In [206]:
import numpy as np
import pandas as pd

In [207]:
# 找到训练数据和测试数据
import os
script_dir = os.getcwd()  # 获取当前的 file 所在的 directory
test_file_path = os.path.join(script_dir,'train.csv')
test_file_path = os.path.join(script_dir,'test.csv')
try:
    test_data = pd.read_csv(test_file_path)
    test_data = pd.read_csv(test_file_path)
    print("训练数据加载成功🎉")
except FileNotFoundError:
    print(f"错误❌:找不到文件 {test_data}。请确保在正确的路径下")

训练数据加载成功🎉


In [208]:
# 处理训练集中的空数据
null_counts_train = test_data.isnull().sum()
print(null_counts_train)

# 使用年龄的中位数填充 Age 列的空值
median_age = test_data["Age"].median()
test_data['Age'] = test_data["Age"].fillna(
    median_age
)  # inplace表示直接在原本的csv文件上进行更改

# cabin缺失的值非常多，可以将 缺失NaN 也作为一种特征值，用字母N表示 ，并将Cabin的值的首字母作为新的值，并删除原来的Cabin
test_data['Cabin'] = test_data['Cabin'].fillna('None')
test_data['Cabin_processed'] = test_data['Cabin'].str[0]
test_data = test_data.drop('Cabin',axis=1)

# 使用 embarked 的众数来表示 空值
mode_embarked = test_data["Embarked"].mode()[0]
test_data['Embarked'] = test_data["Embarked"].fillna(mode_embarked[0])


# 再次检查以确认修改生效
print('\n--- after filling ---')
print(test_data.isnull().sum())


PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

--- after filling ---
PassengerId        0
Survived           0
Pclass             0
Name               0
Sex                0
Age                0
SibSp              0
Parch              0
Ticket             0
Fare               0
Embarked           0
Cabin_processed    0
dtype: int64


In [209]:
# 将不同元素对存活率的影响写入到文件中
output_filename = "feature_analysis.xlsx"
with pd.ExcelWriter(output_filename) as w:
    for element in test_data.columns:
        if element != "Survived":
            single_result = test_data[[element,"Survived"]].groupby(element).mean()
            single_result.to_excel(w,sheet_name = element)
print(f"✅ 不同单一feature的结果已经被写到文件:{output_filename}中")

✅ 不同单一feature的结果已经被写到文件:feature_analysis.xlsx中


In [ ]:
# 构建随机森林模型对数据进行模拟
from sklearn.model_selection import test_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import xgboost as gb

# ---- 1.特征选择与转换 ----
y = test_data["Survived"]
choosed_features = [
    "Age",
    "Sex",
    "Pclass",
    "Embarked",
    "Cabin_processed",
    "SibSp",
    "Parch",
]
# 利用 get_dummies 对 X 使用独热编码，drop_first表示将独热编码数组的第一位舍去
X = pd.get_dummies(test_data[choosed_features], drop_first=True)


# # ---- 2.划分数据集 ----
# X_train,X_val,y_train,y_val = test_test_split(X,y,test_size=0.2,random_state=1)
# print(f"训练集大小{X_train.shape}")
# print(f"测试集大小{X_val.shape}")

# ---- 3.训练模型 ----
model = RandomForestClassifier(n_estimators=500,max_depth=5,random_state=42)
# model = gb.XGBClassifier(n_estimators=500,max_depth=5,learning_rate=0.1,random_state=42)
model.fit(X,y)
print('✅模型训练完成！')

# # ---- 4.评估模型 ----
# predictions = model.predict(X_val)
# accuracy = accuracy_score(y_val,predictions)
# print(f"模型在验证集上的准确度为{accuracy:.4f}")

✅模型训练完成！


In [211]:
# 处理测试集中的数据
null_counts_test = test_data.isnull().sum()
print("====== 处理前的test_data ======")
print(null_counts_test)

# 用中位数填补 age 的空值
test_data["Age"] = test_data["Age"].fillna(median_age)

# 用中位数填补 fare 的空值
median_fare = test_data["Fare"].median()
test_data["Fare"] = test_data["Fare"].fillna(median_fare)

# 处理Cabin（方法同上）
test_data["Cabin"] = test_data["Cabin"].fillna("None")
test_data["Cabin_processed"] = test_data["Cabin"].str[0]
test_data = test_data.drop(labels="Cabin", axis=1)

# 选择与训练时相同的特征值来构建 result_set ，方便后续预测未知结果
result_test = pd.get_dummies(test_data[choosed_features], drop_first=True)

# 超级‼️‼️的一步：校准列
# 举个例子，假如说我的test_data 的 cabin 只有 a,b,c,d,e,f这六种，那它的独热编码也就只有六种
# 但假如说，我的test_data 的 cabin 有 a,b,c,d,e 这五种的话，那它就会多一个特征值，所以此时利用校准列可以将test_data 的特征列完全映射到 test_data 上
X_aligned, result_test_aligned = X.align(result_test, 
                                         join="left",# 表示将result_test沿着左边的那个set，即X进行对齐
                                         axis=1,  # 表示 在列 的维度上进行对齐align
                                         fill_value=0)# fill_value 表示将对齐之后，产生的空位全部替换为 0

print('✅测试数据处理完毕✅')

====== 处理前的test_data ======
PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64
✅测试数据处理完毕✅


In [212]:
# 使用训练好的模型进行预测
print('开始预测...')
result_predictions = model.predict(result_test_aligned)
print('预测完毕')

# 创建提交文件
submission = pd.DataFrame({
    'PassengerId':test_data['PassengerId'],
    'Survived':result_predictions
})

# 保存为 csv 文件
submission.to_csv('submission.csv',index=False)
print('🎉预测的结果文件已经保存')

开始预测...
预测完毕
🎉预测的结果文件已经保存
